# Teste do projeto spot
    - pegar o token
    - extrair os dados da api
    - criar funcoes para extrair os dados da API
    - cria funcao para exportar para mongo

# URL's

    # query = barcelona
        # - 'https://api.foursquare.com/v3/places/search?query=barcelona' 
    # query = caffe - near = barcelona
        - 'https://api.foursquare.com/v3/places/search?query=coffee&near=barcelona' 
    # query = caffe / categories = 11124 Business and Professional Services > Office / near = barcelona
        - 'https://api.foursquare.com/v3/places/search?query=coffee&categories=11124&near=barcelona'     
    # categories = 11124 Business and Professional Services > Office / near = barcelona
        - 'https://api.foursquare.com/v3/places/search?categories=11124&near=barcelona'
    # near = barcelona
        - 'https://api.foursquare.com/v3/places/search?near=barcelona'
    # 12057	Community and Government > Education > Primary and Secondary School
        - https://api.foursquare.com/v3/places/search?query=escola&categories=12057&near=barcelona    
    # 12059	Community and Government > Education > Primary and Secondary School > High School (Tem perto do trablho)
    - https://api.foursquare.com/v3/places/search?query=escola&categories=12059&near=barcelona


# Livrarias

In [1]:
import requests
import json
import geopandas
import cartoframes
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

# API FourSquare 

# Token

In [2]:
load_dotenv()

True

In [3]:
token = getpass.getpass("token: ")


# Funcao Query

In [4]:
def extrator_category_near(cate, near):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "category": f'{cate}',
        'near': f'{near}'
        }
    headers = {
        "Accept": "application/json",
        "Authorization": token
    }
    response = requests.request("GET", url, params=params, headers=headers).json()
    return response

# Funcao Query LAT, LONG

In [5]:
def extrator_query_ll(query,lat, long):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "query": query,
        'll': f'{lat},{long}',
        "sort":"DISTANCE"
        }
    headers = {
        "Accept": "application/json",
        "Authorization": token
    }
    response = requests.request("GET", url, params=params, headers=headers).json()
    return response

# Funcao Categoria, lat, long

In [6]:
def extrator_ll_category(cate,lat, long):
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        'll': f'{lat},{long}',
        'category': f'{cate}',
        "sort":"DISTANCE"
    }
    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    response = requests.request("GET", url, params=params, headers=headers).json()
    return response

In [7]:
def extrator_query_near_category(query, near, cate):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "query": query,
        'near': f'{near}',
        'category': f'{cate}',
        "sort":"DISTANCE"
        }
    headers = {
        "Accept": "application/json",
        "Authorization": token
    }
    response = requests.request("GET", url, params=params, headers=headers).json()
    return response

In [8]:
test_extrator_query_near_category = extrator_query_near_category('school', 'Barcelona', '12057')

In [9]:
test_extrator_category_near = extrator_category_near('12057', 'Barcelona')

In [10]:
test_extrator_query_ll = extrator_query_ll('school', '41.4069929','2.1951855')

In [11]:
test_extrator_ll_category = extrator_ll_category('12057','41.4057403','2.189718')

In [12]:
for result in test_extrator_query_near_category['results']:
    name = result['name']
    address = result['location']['formatted_address']
    latitude = result['geocodes']['main']['latitude']
    longitude = result['geocodes']['main']['longitude']
    print(f"Name: {name} - Address: {address} - Latitude: {latitude}, Longitude: {longitude}")

Name: Button School - Address: Passatge de Mas de Roda, 14, 08005 Barcelona Catalunya - Latitude: 41.388587, Longitude: 2.158134
Name: Global Business School Barcelona - Address: Aragon, 208 (Aribau), 08011 Barcelona Catalunya - Latitude: 41.387934, Longitude: 2.15921
Name: Global Business School Barcelona - Address: Carrer D'Aragó, 208, 08011 Barcelona Barcelona - Latitude: 41.387629, Longitude: 2.159155
Name: Global Business School Barcelona - Address: Carrer D'Aragó, 179, 08011 Barcelona Barcelona - Latitude: 41.387155, Longitude: 2.157936
Name: Barcelona Business School - Address: Calle Valencia, 158, 08011 Barcelona Barcelona - Latitude: 41.386466, Longitude: 2.155026
Name: Barcelona 3d School - Address: Rambla Catalunya, 53, 08007 Barcelona Barcelona - Latitude: 41.390509, Longitude: 2.163853
Name: FX Animation Barcelona 3D School - Address: Pelai, 11, 08001 Barcelona Catalunya - Latitude: 41.390509, Longitude: 2.163853
Name: Virtual Drummer School - Address: Calle Rosellón, 188,

In [13]:
for result in test_extrator_category_near['results']:
    name = result['name']
    address = result['location']['formatted_address']
    latitude = result['geocodes']['main']['latitude']
    longitude = result['geocodes']['main']['longitude']
    print(f"Name: {name} - Address: {address} - Latitude: {latitude}, Longitude: {longitude}")

Name: Roast Club - Address: Carrer de València, 190 (C. de València), 08011 Barcelona Catalunya - Latitude: 41.387825, Longitude: 2.157221
Name: La Central - Address: Calle Mallorca, 237 (C. de Balmes), 08008 Barcelona Catalunya - Latitude: 41.392076, Longitude: 2.159978
Name: Honest Greens Rambla Catalunya - Address: Rambla Catalunya, 3, 08007 Barcelona Catalunya - Latitude: 41.387526, Longitude: 2.167989
Name: Parking Pizza - Address: Calle Londres, 98, 08036 Barcelona Catalunya - Latitude: 41.393794, Longitude: 2.151435
Name: Senyor Vermut - Address: Provença, 85, 08029 Barcelona Catalunya - Latitude: 41.385903, Longitude: 2.149395
Name: La Central del Raval - Address: Carrer D'Elisabets, 6, 08001 Barcelona Catalunya - Latitude: 41.383572, Longitude: 2.168847
Name: Bodega Armando - Address: Calle Bisbe Laguarda, 3, 08001 Barcelona Catalunya - Latitude: 41.3799, Longitude: 2.164802
Name: The Egg Lab - Address: Carrer de Sepúlveda, 80, 08015 Barcelona Catalunya - Latitude: 41.378512, 

In [14]:
for result in test_extrator_query_ll['results']:
    name = result['name']
    address = result['location']['formatted_address']
    latitude = result['geocodes']['main']['latitude']
    longitude = result['geocodes']['main']['longitude']
    print(f"Name: {name} - Address: {address} - Latitude: {latitude}, Longitude: {longitude}")

Name: Foxize School - Address: Llacuna, 162, 08018 Barcelona Catalunya - Latitude: 41.406352, Longitude: 2.192229
Name: Pepe School Land - Address: Passatge Pere Ripoll, 5 (Rambla del Poble Nou con Pere IV), 08005 Barcelona Catalunya - Latitude: 41.403333, Longitude: 2.198838
Name: Old School Bakery Cafe - Address: Bilbao, 216 (Aragó), 08018 Barcelona Catalunya - Latitude: 41.411034, Longitude: 2.191441
Name: Old School - Address: Carrer de Bilbao, 216 (carrer de Aragó), 08018 Barcelona Catalunya - Latitude: 41.411682, Longitude: 2.19192
Name: Chisholm School - Address: Rambla Guipuscoa, 11, 08018 Barcelona Catalunya - Latitude: 41.413524, Longitude: 2.19293
Name: Today School - Address: Calle Valencia, 610, 08026 Barcelona Catalunya - Latitude: 41.409505, Longitude: 2.185992
Name: Urban School - Address: Llull, 187, 08005 Barcelona Catalunya - Latitude: 41.40129, Longitude: 2.202319
Name: Fashion Group Bcn School - Address: Pujades, 142, 08005 Barcelona Catalunya - Latitude: 41.399275

In [15]:
for result in test_extrator_ll_category['results']:
    name = result['name']
    address = result['location']['formatted_address']
    latitude = result['geocodes']['main']['latitude']
    longitude = result['geocodes']['main']['longitude']
    print(f"Name: {name} - Address: {address} - Latitude: {latitude}, Longitude: {longitude}")

Name: Little Fern - Address: Calle Pere IV, 168 (C. Llacuna), 08005 Barcelona Catalunya - Latitude: 41.402328, Longitude: 2.197571
Name: La Tavernícola - Address: Roc Boronat, 70 (Calle De Pujades), 08005 Barcelona Catalunya - Latitude: 41.400358, Longitude: 2.197944
Name: Three Marks Coffee - Address: Calle D'Ausiàs Marc, 151, 08013 Barcelona Catalunya - Latitude: 41.397168, Longitude: 2.183123
Name: Can Dendê - Address: Ciutat de Granada, 44 (C. de Llull), 08005 Barcelona Catalunya - Latitude: 41.39836, Longitude: 2.198345
Name: Dino's Ice Cream - Address: Rambla Poble Nou, 59 (Llul), 08005 Barcelona Catalunya - Latitude: 41.40091, Longitude: 2.201478
Name: Plaça de Gaudí - Address: 08013 Barcelona Catalunya - Latitude: 41.404427, Longitude: 2.175549
Name: Nativity Towers (Torres Naixement) - Address: Calle de Mallorca, 401, Barcelona Catalunya - Latitude: 41.403879, Longitude: 2.17468
Name: The Basilica of the Sagrada Familia (Temple Expiatori de la Sagrada Família) - Address: Carre

In [16]:
def locals(data, cate):
    list_local = []
    df = pd.DataFrame(columns = ['Category','Name', 'Address', 'Latitude', 'Longitude'])
    for result in data['results']:
        local = {}
        local['Category'] = cate
        local['Name'] = result['name']
        local['Address'] = result['location']['formatted_address'] 
        local['Latitude'] = result['geocodes']['main']['latitude']
        local['Longitude'] = result['geocodes']['main']['longitude']
        list_local.append(local)
    df = pd.DataFrame(list_local)
    return df    
        

In [17]:
result_test_extrator_query_ll = locals(test_extrator_query_ll, 'School')
result_test_extrator_query_ll

,Category,Name,Address,Latitude,Longitude
0,School,Foxize School,"Llacuna, 162, 08018 Barcelona Catalunya",41.406352,2.192229
1,School,Pepe School Land,"Passatge Pere Ripoll, 5 (Rambla del Poble Nou ...",41.403333,2.198838
2,School,Old School Bakery Cafe,"Bilbao, 216 (Aragó), 08018 Barcelona Catalunya",41.411034,2.191441
3,School,Old School,"Carrer de Bilbao, 216 (carrer de Aragó), 08018...",41.411682,2.191920
4,School,Chisholm School,"Rambla Guipuscoa, 11, 08018 Barcelona Catalunya",41.413524,2.192930
5,School,Today School,"Calle Valencia, 610, 08026 Barcelona Catalunya",41.409505,2.185992
6,School,Urban School,"Llull, 187, 08005 Barcelona Catalunya",41.401290,2.202319
7,School,Fashion Group Bcn School,"Pujades, 142, 08005 Barcelona Catalunya",41.399275,2.197754
8,School,Complex Esportiu Municipal Bac de Roda,"Rambla Guipúscoa, 25 (Bac de Roda), 08018 Barc...",41.414447,2.194152
9,School,Biblioteca Poblenou-Manuel Arranz,"Joncar, 35, 08005 Barcelona Catalunya",41.401412,2.204491


In [18]:
test_map = Map(location=['41.4046268','2.1893276'], zoom_start=15)

In [19]:
def map_maker(data):
    for i, r in data.iterrows():
        name = r['Name']
        lat = r['Latitude']
        long = r['Longitude']
        color = 'blue'
        icon_color = 'black'
        icon = 'graduation-cap'
        icon = Icon(
                    color = color,
                    icon_color = icon_color,
                    icon = icon,
                    prefix="fa" # font-awesome website: fa
            )
        test_marker = Marker(location=[lat, long], tooltip=name, icon= icon)
        test_marker.add_to(test_map)
    return test_map

In [20]:
school_test = map_maker(result_test_extrator_query_ll)


In [23]:
test_map

# Como enviar o json extraido da api para meu mongo

In [21]:
def export_to_mongodb(data, col):
    # Conectar ao MongoDB (certifique-se de ter o servidor MongoDB em execução)
    client = MongoClient("localhost", 27017)

    # Escolher ou criar um banco de dados e uma coleção no MongoDB
    db = client["Project-Spot"]
    collection = db[col]

    # Inserir os dados no MongoDB
    result = collection.insert_one(data)

    # Imprimir o ID do documento inserido
    print(f"Documento inserido com ID: {result.inserted_id}")

    return result

In [22]:
# Enviar dados extraídos para o MongoDB
export_to_mongodb(api_data, 'coffe shop')

NameError: name 'api_data' is not defined

In [ ]:
# verificar coordenadas
for result in api_data['results']:
    geocodes = result.get('geocodes', {})  # Get the 'geocodes' dictionary, default to an empty dictionary if not present
    for key, coordinates in geocodes.items():
        print(f" coordinates: {coordinates['latitude']}, {coordinates['longitude']}")




 coordinates: 41.384965, 2.164582
 coordinates: 41.384965, 2.164582
 coordinates: 41.381567, 2.159095
 coordinates: 41.381567, 2.159095
 coordinates: 41.398077, 2.162116
 coordinates: 41.398077, 2.162116
 coordinates: 41.400852, 2.17776
 coordinates: 41.400852, 2.17776
 coordinates: 41.376394, 2.160389
 coordinates: 41.376394, 2.160389
 coordinates: 41.373088, 2.174453
 coordinates: 41.373088, 2.174453
 coordinates: 41.402755, 2.183019
 coordinates: 41.402755, 2.183019
 coordinates: 41.391011, 2.147691
 coordinates: 41.391011, 2.147691
 coordinates: 41.40671, 2.155362
 coordinates: 41.407049, 2.155141
 coordinates: 41.407049, 2.155141
 coordinates: 41.39396, 2.202743
 coordinates: 41.39396, 2.202743
